In [1]:
import json
import pandas as pd
import re

In [33]:
df = pd.read_csv('data\schema2\Llama-PSC-Extractor-8B-8bit-Schema-2.csv')
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,full_text,filtered_text,json_output
0,0,2,2,72,Paper #: 72\r\nSuppressed Ion Migration in Red...,T oday's perovskite solar cells (PSCs) require...,"{'perovskite_composition': None, 'electron_tra..."
1,1,3,3,75,Paper #: 75\r\nDynamical Evolution of the 2D/3...,UV-VIS Absorption The absorption spectra of t...,"{\r\n ""perovskite_composition"": null,\r\n ..."
2,2,4,4,40,Paper #: 40\r\nDamp heat-stable perovskite sol...,Damp heat-stable perovskite solar cells with t...,"{'perovskite_composition': 'FAMAPbI3', 'electr..."
3,3,7,7,47,Paper #: 47\r\n\r\nMaterials and Methods \r\n ...,Device fabrication Patterned ITO glass substr...,{'perovskite_composition': 'Cs0.05FA0.81MA0.14...
4,4,8,8,96,Paper #: 96\r\nStable perovskite solar cells u...,Broader context Perovskite solar cells (PSCs)...,"{'perovskite_composition': None, 'electron_tra..."
...,...,...,...,...,...,...,...
82,82,142,142,95,Paper #: 95\r\nOptimal Interfacial Engineering...,DOI: 10.1002/aenm.201902740 perovskites. Becau...,{'perovskite_composition': 'FAPbI3 0.95 (MAPbB...
83,83,145,145,123,Paper #: 123\r\nConstructing heterojunctions b...,P erovskite solar cells (PSCs) have reached po...,{'perovskite_composition': 'FAPbI3.95(MAPbBr3)...
84,84,146,146,31,Paper #: 31\r\n\r\nWe performed 1 H-NMR to sup...,We performed 1 H-NMR to support our proposed r...,{'perovskite_composition': 'Rb0.05Cs0.05MA0.05...
85,85,147,147,36,Paper #: 36\r\nUnderstanding Performance Limit...,Understanding Performance Limiting Interfacial...,"{\r\n ""perovskite_composition"": null, \r\n ..."


In [34]:
df = df[df["json_output"].isna() == False] 

In [35]:
def clean_json(json_string):
    json_string = json_string.replace("None", "null")
    json_string = json_string.replace("'", "\"")
    return json_string

In [40]:
df[df["id"] == 75]["json_output"].iloc[0]

'{\r\n    "perovskite_composition": null,\r\n    "electron_transport_layer": null,\r\n    "pin_nip_structure": null,\r\n    "hole_transport_layer": null,\r\n    "passivating_molecule": null,\r\n    "control_pce": null,\r\n    "control_voc": null,\r\n    "treated_pce": null,\r\n    "treated_voc": null,\r\n    "test_1": {\r\n        "test_name": "ISOS-LT",\r\n        "temperature": null,\r\n        "time": null,\r\n        "humidity": null,\r\n        "retained_percentage_cont": null,\r\n        "retained_percentage_tret": null\r\n    },\r\n    "test_2": {\r\n        "test_name": "ISOS-D",\r\n        "temperature": null,\r\n        "time": null,\r\n        "humidity": null,\r\n        "retained_percentage_cont": null,\r\n        "retained_percentage_tret": null\r\n    },\r\n    "test_3": {\r\n        "test_name": "ISOS-L",\r\n        "temperature": null,\r\n        "time": null,\r\n        "humidity": null,\r\n        "retained_percentage_cont": null,\r\n        "retained_percentage_tret

In [41]:
import re
output = {}
i = 0
for index, row in df.iterrows():
    i += 1
    json_match = re.search(r"\{.*\}", row["json_output"], re.DOTALL)
    if json_match:
        raw_json = json_match.group(0).strip()
    else:
        raw_json = None
    try:
        raw_json = json.loads(clean_json(raw_json))
    except json.JSONDecodeError as e:
        print(row["id"])
        raw_json = None
    output[str(row["id"])] = raw_json

75
127
120
51
18
11
13
148
37
77
132
67
36


In [42]:
output["124"]["perovskite_composition"]

'Cs0.05(FA0.98MA0.02)0.95Pb(I0.98Br0.02)3'

In [43]:
with open('data/schema2/llama_8bit_finetuned.json', 'w') as f:
    json.dump(output, f)